<a href="https://colab.research.google.com/github/alex-pakalniskis/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

# Data Cleaning

In [0]:
import pandas as pd

from scipy import stats

import numpy as np

In [2]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"

data_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv(data_url, names=data_headers)

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [3]:
np.unique(df.values)

array(['?', 'democrat', 'n', 'republican', 'y'], dtype=object)

In [4]:
df = df.replace({"y":1,"n":0,"?":np.NaN})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
republicans = df[df["party"] == "republican"]

republicans.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [6]:
democrats = df[df["party"] == "democrat"]

democrats.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [14]:
issues = df.drop(["party"], axis=1).columns.tolist()

issues

['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free',
 'south-africa']

# Two-sample t-tests

### Functions

In [0]:
def more_democratic_support(democrat_df, 
                            
                            republican_df, 
                            
                            issue_list):

    results = pd.DataFrame(issue_list, 
                         
                         columns=["issues"])
  
    issues_statistic = []     
  
    issues_pvalues = []

    for i in range(len(issue_list)):
  
      issues_statistic.append(stats.ttest_ind(democrat_df[issue_list[i]], 
                                            
                                              republican_df[issue_list[i]], 
                                            
                                              nan_policy="omit").statistic)

      issues_pvalues.append(stats.ttest_ind(democrat_df[issue_list[i]],
                                          
                                            republican_df[issue_list[i]], 
                                          
                                            nan_policy="omit").pvalue)
    
    results["statistic"] = pd.Series(issues_statistic)

    results["pvalue"] = pd.Series(issues_pvalues)

    results["democrat_mean"] = pd.Series(democrat_df.mean().values)

    results["republican_mean"] = pd.Series(republican_df.mean().values)

    more_democratic_support_subset = results[results["democrat_mean"] > results["republican_mean"]]

    confidence_interval_subset = more_democratic_support_subset[more_democratic_support_subset["pvalue"] < 0.01]
    
    return confidence_interval_subset

def more_republican_support(democrat_df, 
                            
                            republican_df, 
                            
                            issue_list):
    
    results = pd.DataFrame(issue_list, 
                         
                         columns=["issues"])
  
    issues_statistic = []     
  
    issues_pvalues = []

    for i in range(len(issue_list)):
  
      issues_statistic.append(stats.ttest_ind(republican_df[issue_list[i]], 
                                            
                                              democrat_df[issue_list[i]], 
                                            
                                              nan_policy="omit").statistic)

      issues_pvalues.append(stats.ttest_ind(republican_df[issue_list[i]],
                                          
                                            democrat_df[issue_list[i]], 
                                          
                                            nan_policy="omit").pvalue)
    
    results["statistic"] = pd.Series(issues_statistic)

    results["pvalue"] = pd.Series(issues_pvalues)

    results["democrat_mean"] = pd.Series(democrat_df.mean().values)

    results["republican_mean"] = pd.Series(republican_df.mean().values)

    more_republican_support_subset = results[results["democrat_mean"] < results["republican_mean"]]

    confidence_interval_subset = more_republican_support_subset[more_republican_support_subset["pvalue"] < 0.01]
    
    return confidence_interval_subset

def not_much_difference(democrat_df, 
                        
                        republican_df, 
                        
                        issue_list):
  
    results = pd.DataFrame(issue_list, 
                         
                         columns=["issues"])
  
    issues_statistic = []     
  
    issues_pvalues = []

    for i in range(len(issue_list)):
  
      issues_statistic.append(stats.ttest_ind(democrat_df[issue_list[i]], 
                                            
                                              republican_df[issue_list[i]], 
                                            
                                              nan_policy="omit").statistic)

      issues_pvalues.append(stats.ttest_ind(democrat_df[issue_list[i]],
                                          
                                            republican_df[issue_list[i]], 
                                          
                                            nan_policy="omit").pvalue)
    
    results["statistic"] = pd.Series(issues_statistic)

    results["pvalue"] = pd.Series(issues_pvalues)

    results["democrat_mean"] = pd.Series(democrat_df.mean().values)

    results["republican_mean"] = pd.Series(republican_df.mean().values)

    not_much_difference_subset = results[results["pvalue"] > 0.1]
    
    return not_much_difference_subset


### Hypothesis testing

**Null-hypothesis**: Democrat support for a bill is equal to republican support for a bill.

$H_0: \bar{x}_{Republican} == \bar{x}_{Democrat}$


**Alternative hypothesis**: Democrat support for a bill is not equal to Republican support for a bill

$H_a: \bar{x}_{Republican} \neq \bar{x}_{Democrat}$

Confidence level: 99%

In [106]:
more_democratic_support(democrats, republicans, issues)

,issues,statistic,pvalue,democrat_mean,republican_mean
0,handicapped-infants,9.205264,1.613440e-18,0.604651,0.187879
2,budget,23.212777,2.070340e-77,0.888462,0.134146
6,anti-satellite-ban,12.526188,8.521033e-31,0.772201,0.240741
7,aid-to-contras,18.052093,2.824718e-54,0.828897,0.152866
8,mx-missile,16.437503,5.030793e-47,0.758065,0.115152
10,synfuels,8.293604,1.575932e-15,0.505882,0.132075
14,duty-free,12.853146,5.997697e-32,0.637450,0.089744
15,south-africa,6.849455,3.652674e-11,0.935135,0.657534


**Conclusion A**: Based on positive t-statistic values > 0 and p-values < 0.01, I reject the null hypothesis that democrats and republicans voted for these issues with equal support. In contrast, I suggest the alternative hypothesis that democrats and republicans exhibited different voting support for these issues. As evidenced by the above data, democrat support was higher for "handicapped-infants", "budget", "anti-satellite-ban", "aid-to-contras", "mx-missile", "synfuels", "duty-free", and "south-africa" issues in 1984. 

In [121]:
more_republican_support(democrats, republicans, issues)

,issues,statistic,pvalue,democrat_mean,republican_mean
3,physician-fee-freeze,49.367082,1.994262e-177,0.054054,0.987879
4,el-salvador-aid,21.136693,5.600520e-68,0.215686,0.951515
5,religious-groups,9.737576,2.393672e-20,0.476744,0.897590
11,education,20.500686,1.883420e-64,0.144578,0.870968
12,right-to-sue,13.510643,1.227858e-34,0.289683,0.860759
13,crime,16.342086,9.952343e-47,0.350195,0.981366


**Conclusion B**: Based on positive t-statistic values > 0 and p-values < 0.01, I reject the null hypothesis that democrats and republicans voted for these issues with equal support. In contrast, I suggest the alternative hypothesis that democrats and republicans exhibited different voting support for these issues. As evidenced by the above data, republican support was higher for "physician-fee-freeze", "el-salvador-aid", "religious-groups", "education', "right-to-sue", and "crime" issues in 1984. 

In [122]:
not_much_difference(democrats, republicans, issues)

,issues,statistic,pvalue,democrat_mean,republican_mean
1,water-project,-0.088965,0.929156,0.502092,0.506757


**Conclusion C**: Based on near-zero t-statistic values and p-values > 0.1, I accept the null hypothesis that democrats and republicans voted for these issues with equal support.  As evidenced by the above data, republican and democratic support was equal for "water-project" issues in 1984. 